In [134]:
# Hadley Wickham - coined the term 'split-apply-combine' for describing group operations

In [135]:
import pandas as pd
import numpy as np

In [136]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.919477,0.732518
1,a,two,1.225433,-2.161366
2,b,one,-1.723126,0.097349
3,b,two,-0.485698,0.402637
4,a,one,0.627803,1.738055


In [137]:
# compute mean of the data1 column using the labels from key1
# access data1 and call groupby with the column which is a Series at key1

In [138]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [139]:
# the above grouped variable is now a GroupBy object
# this object has not computed anything yet
# only intermediate data about the group key df['key'] is inferred here
# this object has all the information needed to apply some ops to each of the groups

In [140]:
grouped.mean()

key1
a    0.311253
b   -1.104412
Name: data1, dtype: float64

In [141]:
# here the data has been aggregated as per the group key
# results in a new Series indexed by the unique values in key1 column
# result index has name 'key1' as the DF column df['key1'] has the same name

In [142]:
# pass multiple arrays as a list
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.145837
      two     1.225433
b     one    -1.723126
      two    -0.485698
Name: data1, dtype: float64

In [143]:
# result Series has a hierarchical index with unique pair of keys
means.unstack()

key2,one,two
key1,,
a,-0.145837,1.225433
b,-1.723126,-0.485698


In [144]:
# using group keys as arrays of the right length
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [145]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [146]:
df['data1'].groupby([states, years]).mean()

California  2005    1.225433
            2006   -1.723126
Ohio        2005   -0.702588
            2006    0.627803
Name: data1, dtype: float64

In [147]:
# when grouping information is found in the same DF as the data
# pass column names as the group key
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.311253,0.103069
b,-1.104412,0.249993


In [148]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.145837  1.235286
     two   1.225433 -2.161366
b    one  -1.723126  0.097349
     two  -0.485698  0.402637

In [149]:
# size - returns a Series containing group sizes
# returns the count/size of similar elements
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [150]:
# missing values in a group key will be excluded from the result

### ITERATING OVER GROUPS

In [151]:
# GroupBy supports iteration, generating sequence of 2-tuples
# tuples contain the group name and chunk of data

In [152]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.919477  0.732518
1    a  two  1.225433 -2.161366
4    a  one  0.627803  1.738055
b
  key1 key2     data1     data2
2    b  one -1.723126  0.097349
3    b  two -0.485698  0.402637


In [153]:
# in case of multiple keys, 1st element in the tuple will be a tuple of key values
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.919477  0.732518
4    a  one  0.627803  1.738055
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.225433 -2.161366
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.723126  0.097349
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.485698  0.402637


In [154]:
# computing a dict of data pieces as a one-liner
pieces = dict(list(df.groupby('key1')))

In [155]:
pieces['a']

,key1,key2,data1,data2
0,a,one,-0.919477,0.732518
1,a,two,1.225433,-2.161366
4,a,one,0.627803,1.738055


In [156]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.723126,0.097349
3,b,two,-0.485698,0.402637


In [157]:
# groupby groups on axis=0
# grouping the columns by dtype
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [158]:
grouped = df.groupby(df.dtypes, axis=1)

In [159]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.919477  0.732518
1  1.225433 -2.161366
2 -1.723126  0.097349
3 -0.485698  0.402637
4  0.627803  1.738055
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### SELECTING A COLUMN OR SUBSET OF COLUMNS

In [160]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.235286
     two  -2.161366
b    one   0.097349
     two   0.402637

In [161]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [162]:
s_grouped.mean()

key1  key2
a     one     1.235286
      two    -2.161366
b     one     0.097349
      two     0.402637
Name: data2, dtype: float64

### GROUPING WITH DICTS AND SERIES

In [163]:
# grouping information may exist in a form other than array

In [164]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,-2.279986,-0.530291,2.025582,-0.091743,-0.309215
Steve,-2.975381,0.134832,-0.228482,0.401846,0.399164
Wes,0.476775,-1.014571,0.301847,-0.312608,-1.670428
Jim,0.873789,0.088030,0.006448,-1.461305,-1.005858
Travis,-1.191759,0.613674,0.337136,0.360785,-0.240611


In [165]:
# adding few NA values
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-2.279986,-0.530291,2.025582,-0.091743,-0.309215
Steve,-2.975381,0.134832,-0.228482,0.401846,0.399164
Wes,0.476775,NaN,NaN,-0.312608,-1.670428
Jim,0.873789,0.088030,0.006448,-1.461305,-1.005858
Travis,-1.191759,0.613674,0.337136,0.360785,-0.240611


In [166]:
# using a group correspondence for columns, sum together the columns by group
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}

In [167]:
# construct an array from the above dict
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,1.933840,-3.119492
Steve,0.173363,-2.441385
Wes,-0.312608,-1.193652
Jim,-1.454857,-0.044040
Travis,0.697920,-0.818696


In [168]:
# same funtionalities hold for Series, viewed as a fixed-size mapping
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [169]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### GROUPING WITH FUNCTIONS

In [170]:
# using Python functions is more generic way of defining a group mapping compared with dict or Series
# any function passed as a group key will be called once per index value
# return values will be used as group names

In [171]:
# group by length of the names
people.groupby(len).sum()

,a,b,c,d,e
3,-0.929422,-0.442261,2.032030,-1.865656,-2.985501
5,-2.975381,0.134832,-0.228482,0.401846,0.399164
6,-1.191759,0.613674,0.337136,0.360785,-0.240611


In [172]:
# mixing functions with arrays, dicts or Series gets converted to arrays internally
key_list = ['one', 'one', 'one', 'two', 'two']

In [173]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -2.279986 -0.530291  2.025582 -0.312608 -1.670428
  two  0.873789  0.088030  0.006448 -1.461305 -1.005858
5 one -2.975381  0.134832 -0.228482  0.401846  0.399164
6 two -1.191759  0.613674  0.337136  0.360785 -0.240611

### GROUPING BY INDEX LEVELS

In [174]:
# convenience for hierarchical indexed dataset is to aggregate using one of the levels of an axis index

In [175]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

In [176]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.829488 -0.305409 -0.798598 -0.816004 -0.412681
1      0.135181 -0.217881 -0.654779 -0.937569  1.463127
2      0.625321 -0.934925 -0.479572  1.303068  0.216143
3      1.145658  0.814532  0.345634  1.304592  1.319589

In [177]:
# to group by level, pass the level number or name using the level keyword
hier_df.groupby(level="cty", axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## DATA AGGREGATION

In [178]:
# refers to any data transformation that produces scalar values from arrays
# examples - mean, count, min, sum, etc.
# aggregations can be called on any method that is defined on the grouped object
# example - quantiles computes sample quantiles of a Series or a DF's columns

In [179]:
df

,key1,key2,data1,data2
0,a,one,-0.919477,0.732518
1,a,two,1.225433,-2.161366
2,b,one,-1.723126,0.097349
3,b,two,-0.485698,0.402637
4,a,one,0.627803,1.738055


In [180]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.105907
b   -0.609441
Name: data1, dtype: float64

In [181]:
# to use own aggregation functions, pass any function that aggregates an array to the method
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [182]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.144910,3.899421
b,1.237428,0.305289


In [183]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.311253  1.106938 -0.919477 -0.145837  0.627803  0.926618   
b      2.0 -1.104412  0.874994 -1.723126 -1.413769 -1.104412 -0.795055   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.225433   3.0  0.103069  2.024481 -2.161366 -0.714424  0.732518   
b    -0.485698   2.0  0.249993  0.215872  0.097349  0.173671  0.249993   

                          
           75%       max  
key1                      
a     1.235286  1.738055  
b     0.326315  0.402637

### COLUMN-WISE AND MULTIPLE FUNCTION APPLICATION

In [184]:
# load tipping dataset and add a tipping percentage column tip_pct
tips = pd.read_csv('../resources/examples/tips.csv')

In [185]:
# add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [186]:
# easy to aggregate a Series or all columns of DF using aggregate with desired function
# aggregating using a different function on specific column, or using multiple functions at once requires more efforts but doable

In [187]:
# grouping tips by day and smoker
grouped = tips.groupby(['day', 'smoker'])

In [188]:
# for descriptive stats, pass the name of the function as a string
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [189]:
# passing a list of functions will return a DF with column names taken from the functions
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [190]:
# pass a list of (name, function) tuples to be used as the DF column names
# list of 2-tuples as an ordered mapping
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [191]:
# DF has more options, as you can specify a list of functions to apply to all of the columns or different functions per column

In [192]:
functions = ['count', 'mean', 'max']

In [193]:
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [194]:
# the resulting DF has hierarchical columns 
# similar to aggregating each column separately and using concat to glue the results together using column names as key args
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [195]:
# a list of tuples with custom names can be passed

In [196]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [197]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [198]:
# to pass different functions to one or more of the columns
# pass a dict to agg that contains a mapping of column names to any function
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [199]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
             'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [200]:
# DF will have hierarchical columns only if multiple functions are applied to at least one column

### RETURNING AGGREGATED DATA WITHOUT ROW INDEXES

In [201]:
# aggregated data comes back with an index which is hierarchical and composed from unique group key combinations
# this can be disabled by passing as_index=False to groupby
# similar can be achieved by reset_index on the result
# using as_index=False method avoids unnecessary computations

In [202]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


### APPLY: GENERAL SPLIT-APPLY-COMBINE

In [203]:
# most general purpose GroupBy method is apply
# apply splits the object being manipulated into pieces, then attempts to concatenate the pieces together

In [204]:
# from tipping dataset select the top five tip_pct values by group
# write a function that selects the rows with the largest values in a particular column
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [205]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [206]:
# group by smoker and call apply with this function
# top function is called on each row group from the DF
# results are glued together using pandas.concat
# pieces are labelled with group names
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [207]:
# functions which take in args can be passed after the function
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [208]:
# note: apply method requires creativity when passing the arg

In [209]:
# describe on a GroupBy object
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [210]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [211]:
# the describe call on GroupBy object is a shortcut for:
# f = lambda x: x.describe()
# grouped.apply(f)

### SUPPRESSING THE GROUP KEYS

In [212]:
# the above resulting object has a hierarchical index 
# formed from the group keys along with indexes of each piece of original object
# disable this by:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### QUANTILE AND BUCKET ANALYSIS

In [213]:
# cut and qcut - slicing data up into buckets with bins of sample quantiles

In [214]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)

In [215]:
quartiles[:10]

0      (-0.0742, 1.48]
1      (-0.0742, 1.48]
2      (-0.0742, 1.48]
3    (-1.629, -0.0742]
4      (-0.0742, 1.48]
5      (-0.0742, 1.48]
6        (1.48, 3.035]
7    (-1.629, -0.0742]
8      (-0.0742, 1.48]
9      (-0.0742, 1.48]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.189, -1.629] < (-1.629, -0.0742] < (-0.0742, 1.48] < (1.48, 3.035]]

In [216]:
# the categorical object returned by cut can be passed to groupby
# compute a set of stats for data2 column:
def get_stats(group):
    return {'min': group.min(),
            'max': group.max(),
            'count': group.count(),
            'mean': group.mean()}

In [217]:
grouped = frame.data2.groupby(quartiles)
grouped

In [218]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.189, -1.629]",40.0,2.777946,-0.131175,-1.524146
"(-1.629, -0.0742]",455.0,3.226359,-0.005679,-2.911636
"(-0.0742, 1.48]",442.0,2.698478,0.057760,-2.784097
"(1.48, 3.035]",63.0,1.997122,0.044588,-1.879029


In [219]:
# to return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

In [220]:
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.777946,0.046630,-2.364529
1,100.0,3.059501,-0.015492,-2.105278
2,100.0,2.204801,0.014645,-2.550046
3,100.0,2.497102,-0.079933,-2.851947
4,100.0,3.226359,-0.030286,-2.911636
5,100.0,1.897637,0.138263,-2.181467
6,100.0,2.206008,0.113901,-2.353647
7,100.0,2.416471,-0.052734,-2.284504
8,100.0,2.698478,0.045398,-2.417785


### Example: Filling missing values with Group-Specific values

In [221]:
# for cleaning up missing data, usual step would be to use dropna
# to impute (fillna) the null (NA) values using fixed value or some value from data
# fillna serves this purpose

# below is the approach to fill NA values with mean
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.155906
2         NaN
3    0.493932
4         NaN
5    0.511066
dtype: float64

In [222]:
s.fillna(s.mean())

0    0.283030
1   -0.155906
2    0.283030
3    0.493932
4    0.283030
5    0.511066
dtype: float64

In [223]:
# fill value to vary by group
# group data and use apply with function that calls fillna on each data chunk

In [224]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon',
          'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4

In [225]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.697288
New York      0.485446
Vermont       2.499371
Florida       0.362669
Oregon       -2.044948
Nevada        0.608554
California   -0.221986
Idaho         0.001871
dtype: float64

In [226]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.697288
New York      0.485446
Vermont            NaN
Florida       0.362669
Oregon       -2.044948
Nevada             NaN
California   -0.221986
Idaho              NaN
dtype: float64

In [227]:
data.groupby(group_key).mean()

East    0.515135
West   -1.133467
dtype: float64

In [228]:
# now we can fill the NA values using the group means
fill_mean = lambda x: x.fillna(x.mean())

In [229]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.697288
New York      0.485446
Vermont       0.515135
Florida       0.362669
Oregon       -2.044948
Nevada       -1.133467
California   -0.221986
Idaho        -1.133467
dtype: float64

In [230]:
# if the fill values are predefined in the code that vary by group
# use the name attribute of groups which are set internally
fill_values = {'East': 0.5, 'West': -1}

fill_func = lambda x: x.fillna(fill_values[x.name])

In [231]:
data.groupby(group_key).apply(fill_func)

Ohio          0.697288
New York      0.485446
Vermont       0.500000
Florida       0.362669
Oregon       -2.044948
Nevada       -1.000000
California   -0.221986
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [232]:
# draw a random sample from a large dataset for Monte Carlo simulation
# number of ways to perform draws, use sample method of Series

In [233]:
# construct a deck of playing cards as an example
# result will be a Series of length 52 whose index contains card names and values
# ace 'A' is kept as 1

In [234]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']

cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

In [235]:
deck = pd.Series(card_val, index=cards)

In [236]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [237]:
# random drawing a hand of five cards from the deck
def draw(deck, n=5):
    return deck.sample(n)

In [238]:
draw(deck)

7S    7
5S    5
4S    4
6S    6
7D    7
dtype: int64

In [239]:
# draw two random cards from each suit
# now as the suit is the last character of each card name, we can group based on this and use apply
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  AC      1
   10C    10
D  QD     10
   7D      7
H  JH     10
   KH     10
S  7S      7
   5S      5
dtype: int64

In [240]:
# alternate way to write this
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

JC    10
KC    10
JD    10
6D     6
5H     5
6H     6
KS    10
QS    10
dtype: int64

### Example: Group Weighted Average and Correlation

In [241]:
# group weighted average is possible b/w columns in DF or Series

In [242]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.randn(8)})
df

,category,data,weights
0,a,-0.620318,-0.263089
1,a,0.842979,-1.682205
2,a,0.532748,1.168092
3,a,0.627740,0.992739
4,b,0.458493,-0.212765
5,b,-0.233263,2.089419
6,b,-0.701399,-1.015543
7,b,1.102338,-0.804393


In [243]:
# group weighted average by category would be
grouped = df.groupby('category')
get_wavg = lambda x: np.average(x['data'], weights=x['weights'])

In [244]:
grouped.apply(get_wavg)

category
a    -0.043538
b   -13.388126
dtype: float64

In [245]:
# dataset example - financial dataset from Yahoo!
# contains end-of-day prices for a few stocks and the S&P500 index

In [246]:
close_px = pd.read_csv('../resources/examples/stock_px_2.csv',
                       parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [247]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [248]:
# task to perform would be to compute DF of yearly correlations of daily returns with SPX
# create a function that computes the pairwise correlation of each column with the 'SPX' column

In [249]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [250]:
# compute percent change on close_px using pct_change
rets = close_px.pct_change().dropna()
rets.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [251]:
# group the percent changes by year, extracted from each row label with one-line function
# returns the year attribute of each datetime label
get_year = lambda x: x.year

by_year = rets.groupby(get_year)

by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [252]:
# compute inter-column correlations b/w Apple and Microsoft
by_year.apply(lambda x: x['AAPL'].corr(x['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [253]:
# groupby can be used to perform complex group-wise stat analysis
# works as long as returned object is a pandas object or scalar value

In [254]:
# performing regress which executes an ordinary least squares regression on each chunk of data

In [255]:
import statsmodels.api as sm

In [259]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

In [260]:
# run a yearly linear regression of AAPL on SPX returns
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


### PIVOT TABLES AND CROSS-TABULATION

In [263]:
# pivot table - data summarization tool frequency
# aggregates a table of data by one or more keys, arranging data in rectangle with some of the group keys along with rows and some along the columns
# pivot tables are possible with pandas through the groupby facility with reshape ops utilizing hierarchical indexing

In [264]:
# pivot_table can add partial totals, known as margins

In [265]:
# compute a table of group means, default pivot_table aggregation arranged by day and smoker on the rows
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [266]:
# above can be produced by groupby directly
# to aggregate only tip_pct and size with grouping by time (smoker in table columns and day in the rows)

In [269]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [270]:
# augment table to include partial totals by passing margins=True
tips.pivot_table(['tip_pct', 'size'],
                 index=['time', 'day'],
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [271]:
# All column depicts mean calculated without taking into account smoker v/s non-smoker

In [274]:
# using different aggregation function
# count or len will give cross-tabulation of group sizes
tips.pivot_table('tip_pct',
                 index=['time', 'smoker'],
                 columns='day', aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [275]:
# NA values can be filled with fill_value
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### CROSS-TABULATIONS: CROSSTAB

In [278]:
# a special case of pivot table that computes group frequencies

In [279]:
# first two args to crosstab can each either be an array or Series or a list of arrays
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244